## --- Day 17: Trick Shot ---
- The probe's x position increases by its x velocity.
- The probe's y position increases by its y velocity.
- Due to drag, the probe's x velocity changes by 1 toward the value 0; that is, it decreases by 1 if it is greater than 0, increases by 1 if it is less than 0, or does not change if it is already 0.
- Due to gravity, the probe's y velocity decreases by 1.

Find the initial velocity that causes the probe to reach the highest y position and still eventually be within the target area after any step. What is the highest y position it reaches on this trajectory?

Helpful:
__[/r/adventofcode: Never brute force when you can use math](https://www.reddit.com/r/adventofcode/comments/rily4v/2021_day_17_part_2_never_brute_force_when_you_can/)__


In [1]:
def find_max_vy(y_bounds):
    return abs(min(y_bounds)) - 1

def find_max_height(y_bounds):
    max_vy = find_max_vy(y_bounds)
    max_height = max_vy * (max_vy + 1) / 2

    return int(max_height)

In [2]:
# Example
ex1_y_input = (-10, -5)
assert 45 == find_max_height(ex1_y_input)

In [3]:
# Part 1
p1_y_input = (-102, -78)
find_max_height(p1_y_input)

5151

## --- Part Two ---

How many distinct initial velocity values cause the probe to be within the target area after any step?

In [4]:
from collections import defaultdict
from itertools import product

def y_pos_after_step(vy, step):
    if step == 0:
        return 0

    drag = (step*(step-1)//2)
    return step * vy - drag

def find_step_vy(ymin, ymax):
    step_vy = defaultdict(list)
    vy_min = ymin
    vy_max = find_max_vy((ymin, ymax))

    for vy in range(vy_min, vy_max+1):
        step, y_pos = 1, 0
        while y_pos >= ymin:
            y_pos = y_pos_after_step(vy, step)
            if y_pos >= ymin and y_pos <= ymax:
                step_vy[step].append(vy)
            step += 1

    return step_vy

def x_pos_after_step(vx, step):
    if step == 0:
        return 0

    if step < vx:
        drag = (step*(step-1)//2)
        return step * vx - drag
    else:
        # dx is constant after step >= vx because drag has overcome initial vx
        drag = (vx*(vx-1)//2)
        return vx**2 - drag

def find_step_vx(xmin, xmax, y_steps):
    steps = defaultdict(list)
    # Maximum possible vx is xmax, otherwise out of range after s1
    for vx, step in product(range(xmax+1), y_steps):
        xpos = x_pos_after_step(vx, step)
        if xpos >= xmin and xpos <= xmax:
            steps[step].append(vx)

            if xpos == vx+1:
                steps[step].append(float('inf'))

    return steps

def find_vx_vy_pairs(xmin, xmax, ymin, ymax):
    step_vy = find_step_vy(ymin, ymax)
    step_vx = find_step_vx(xmin, xmax, y_steps=step_vy.keys())
    
    # Combine all pairs with matching step values
    pairs = set()

    for step in step_vy.keys():
        pairs.update([(vx, vy) for vx, vy in product(step_vx[step], step_vy[step])])

    return pairs


In [5]:
# Example (same inputs)
ex1_x_input = (20, 30)

ex2_pairs = find_vx_vy_pairs(*ex1_x_input, *ex1_y_input)
assert 112 == len(ex2_pairs)

In [6]:
# Part 2 solution
p2_pairs = find_vx_vy_pairs(135, 155, *p1_y_input)
len(p2_pairs)

968